# How to use this notebook

This notebook is meant to be run in a VertexAI Workbench. Here's how to start a VertexAI Workbench notebook:

https://bluestate.atlassian.net/wiki/spaces/ATeam/pages/2983624708/How+to+start+a+VertexAI+Workbench


You'll need to clone this notebook into VertexAI. Here's how to clone in Vertex AI:

https://cloud.google.com/vertex-ai/docs/workbench/user-managed/save-to-github#:~:text=In%20your%20GitHub%20repository%2C%20click,Click%20the%20Git%20clone%20button.

In [ ]:
from google.cloud import bigquery
from time import sleep
# Construct a BigQuery client object.
client = bigquery.Client()

In [ ]:
project_id = input("Please enter the project you'd like to work in\n")

In [ ]:
proceed = True
while proceed:
    origin_schema = input("Please enter the schema (called 'dataset' in BigQuery) you'd like to clone from:\n")
    target_schema = input("Please enter the schema (called 'dataset' in BigQuery) you'd like to clone to:\n")
    dataset_id = f'{project_id}.{origin_schema}'

    tables = client.list_tables(dataset_id)  # Make an API request.
    qs = []
    print("Cloning tables contained in '{}':\n".format(dataset_id))
    for table in tables:
        print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))
        if table.table_type == 'VIEW':
            clone_table_statement=f"""
            CREATE OR REPLACE VIEW
              {project_id}.{target_schema}.{table.table_id}
              AS SELECT * FROM {project_id}.{origin_schema}.{table.table_id}
            """
        else:
            clone_table_statement=f"""
            CREATE OR REPLACE TABLE
              {project_id}.{target_schema}.{table.table_id}
              CLONE {project_id}.{origin_schema}.{table.table_id}
            """
        qs.append(client.query(clone_table_statement))
        print(f"Started cloning to {project_id}.{target_schema}.{table.table_id}\n")
    num_queries = len(qs)
    num_complete = sum([q.running()==False for q in qs])
    while num_complete < num_queries:
        print(f"{num_complete} out of {num_queries} completed!")
        sleep(10)
        num_complete = sum([q.running()==False for q in qs])
    print('\n-------------------------------------------------------------------------------------------------')
    proceed = input("\nIf you'd like to clone another schema please enter 'yes'; only the full word yes will be accepted")=='yes'